In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

# Extracting Information from Text Data Assignment

In [51]:
!pip install textacy

<IPython.core.display.Javascript object>

In [6]:
!pip install rake_nltk

<IPython.core.display.Javascript object>

In [52]:
import spacy
import string
import itertools
import requests
import textacy
from nltk import pos_tag
from rake_nltk import Rake
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import tree2conlltags
from gensim.summarization import keywords
from nltk.chunk.regexp import RegexpParser
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from bs4 import BeautifulSoup

<IPython.core.display.Javascript object>

### Read the CNN Lite plain text file articles into a corpus using the NLTK's PlaintextCorpusReader.

In [27]:
articles = [
    "http://lite.cnn.io/en/article/h_eac18760a7a7f9a1bf33616f1c4a336d",
    "http://lite.cnn.io/en/article/h_de3f82f17d289680dd2b47c6413ebe7c",
    "http://lite.cnn.io/en/article/h_72f4dc9d6f35458a89af014b62e625ad",
    "http://lite.cnn.io/en/article/h_aa21fe6bf176071cb49e09d422c3adf0",
    "http://lite.cnn.io/en/article/h_8ad34a532921c9076cdc9d7390d2f1bc",
    "http://lite.cnn.io/en/article/h_84422c79110d9989177cfaf1c5f45fe7",
    "http://lite.cnn.io/en/article/h_d010d9580abac3a44c6181ec6fb63d58",
    "http://lite.cnn.io/en/article/h_fb11f4e9d7c5323e75b337d9e9e5e368",
    "http://lite.cnn.io/en/article/h_7b27f0b131067f8ece6238ac559670ab",
    "http://lite.cnn.io/en/article/h_8cae7f735fa9573d470f802063ceffe2",
    "http://lite.cnn.io/en/article/h_72c3668280e82576fcc2602b0fa70c14",
    "http://lite.cnn.io/en/article/h_d20658fb0e20212051cda0e0a7248c8a",
    "http://lite.cnn.io/en/article/h_56611c43d7928120d2ae21666ccc7417",
    "http://lite.cnn.io/en/article/h_bda0394e3c5ee7054ee65c022bca7695",
]

<IPython.core.display.Javascript object>

In [28]:
def html_to_text(html):
    TAGS = ["h1", "h2", "h3", "h4", "h5", "h6", "h7", "p", "li"]

    soup = BeautifulSoup(html, "lxml")
    text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
    text = " ".join(text_list)
    return text

<IPython.core.display.Javascript object>

In [29]:
!mkdir news_articles

A subdirectory or file news_articles already exists.


<IPython.core.display.Javascript object>

In [31]:
path = "./news_articles/"

for index, url in enumerate(articles):
    response = requests.get(url)
    html = response.text
    text = html_to_text(html)

    with open(path + "article_" + str(index) + ".txt", "wb") as f:
        f.write(text.encode())

<IPython.core.display.Javascript object>

In [32]:
DOC_PATTERN = r".*\.txt"
news_corpus = PlaintextCorpusReader("news_articles", DOC_PATTERN)

<IPython.core.display.Javascript object>

In [34]:
def corpus_stats(corpus):
    print("Corpus Statistics")
    print("Number of documents: " + str(len(corpus.fileids())))
    print("Number of paragraphs: " + str(len(corpus.paras())))
    print("Number of sentences: " + str(len(corpus.sents())))
    print("Number of words: " + str(len(corpus.words())))
    print("Vocabulary: " + str(len(set(w.lower() for w in corpus.words()))))
    print(
        "Avg chars per word: " + str(round(len(corpus.raw()) / len(corpus.words()), 1))
    )
    print(
        "Avg words per sentence: "
        + str(round(len(corpus.words()) / len(corpus.sents()), 1))
    )


corpus_stats(news_corpus)

Corpus Statistics
Number of documents: 14
Number of paragraphs: 14
Number of sentences: 590
Number of words: 14159
Vocabulary: 2914
Avg chars per word: 5.0
Avg words per sentence: 24.0


<IPython.core.display.Javascript object>

### Iterate through the fileids in the corpus, extract the raw text of each document, and store them in a list.

In [35]:
docs = [news_corpus.raw(file_id) for file_id in news_corpus.fileids()]

<IPython.core.display.Javascript object>

### Extract the top 5 keywords from every document in the corpus. Print them and compare the differences in keywords among the documents.

In [40]:
all_key_words = []
for doc in docs:
    key_words = keywords(doc, words=5, lemmatize=True).split("\n")
    all_key_words.append(key_words)

<IPython.core.display.Javascript object>

In [41]:
all_key_words

[['pink', 'cnn', 'carey', 'tour', 'pretty'],
 ['patrick', 'primary', 'cnn', 'news', 'telling'],
 ['narwhal', 'tail', 'puppy', 'cnn', 'unicorn'],
 ['states', 'democratic', 'news', 'bloomberg', 'cnn'],
 ['republican', 'taylor', 'presidents', 'rep', 'ukraine'],
 ['muslimness', 'people', 'skin', 'white', 'religion'],
 ['news', 'trump', 'said', 'republican', 'media'],
 ['said', 'brown', 'jones', 'police', 'roanoke'],
 ['trump', 'hotels', 'office', 'profit', 'owned'],
 ['keys', 'grammys', 'award', 'reserved', 'wanna'],
 ['americans', 'republican', 'trump', 'ukrainians', 'investigate'],
 ['student', 'said', 'told', 'pence', 'schools'],
 ['cnn', 'crows', 'disney', 'american', 'old'],
 ['protester', 'police', 'news', 'chinese', 'kong']]

<IPython.core.display.Javascript object>

### Extract the top 3 keyphrases from each document, print them, and compare the differences.

In [53]:
top_3_keyphrases = []
all_key_phrases = []
r = Rake()
for doc in docs:
    r.extract_keywords_from_text(doc)
    key_phrases = r.get_ranked_phrases_with_scores()
    all_key_phrases.append(key_phrases)
    top_3_keyphrases.append(key_phrases[0:3])

<IPython.core.display.Javascript object>

In [48]:
top_3_keyphrases

[[(32.5, 'country music association awards red carpet'),
  (25.0, '© 2019 cable news network'),
  (17.166666666666664, 'school soon ," pink said')],
 [(94.03333333333333,
   'former new york city mayor michael bloomberg stepped forward last week'),
  (30.61111111111111, 'patrick could seize upon potential advantages'),
  (30.0, 'elections process would ultimately splash back')],
 [(28.166666666666668, 'little magical furry unicorn ," according'),
  (25.0, '© 2019 cable news network'),
  (17.666666666666664, 'dog rescue nonprofit organization mac')],
 [(40.45, 'current 2020 democrats -- clinton told bbc radio'),
  (37.58333333333333, 'former new york mayor michael bloomberg made'),
  (29.0, '2016 democratic nominee playfully tweeted back')],
 [(69.33333333333334,
   'money ," tweeted white house press secretary stephanie grisham'),
  (67.66666666666667,
   'former white house homeland security adviser tom bossert summed'),
  (63.416666666666664,
   'former national security council russ

<IPython.core.display.Javascript object>

### Identify and extract the named entities in each document, filtering out the numeric types. Print them and compare the differences between documents.

In [49]:
all_entities = []
for doc in docs:
    nlp = spacy.load("en_core_web_sm")
    spacy_doc = nlp(doc)
    entities = [[entity.text, entity.label_] for entity in spacy_doc.ents]
    all_entities.append(entities)

<IPython.core.display.Javascript object>

In [50]:
all_entities

[[['CNN', 'ORG'],
  ['2020', 'DATE'],
  ['Entertainment Tonight', 'WORK_OF_ART'],
  ['the Country Music Association Awards', 'ORG'],
  ['Carey Hart', 'PERSON'],
  ['Willow', 'PERSON'],
  ['8', 'DATE'],
  ['Jameson', 'PERSON'],
  ['2', 'DATE'],
  ['Love Me Anyway', 'WORK_OF_ART'],
  ['Chris Stapleton', 'PERSON'],
  ['two and a half years', 'DATE'],
  ['Willow', 'PERSON'],
  ['Jameson', 'PERSON'],
  ['Pink', 'ORG'],
  ['the year', 'DATE'],
  ['14 years', 'DATE'],
  ['January', 'DATE'],
  ['Carey', 'PERSON'],
  ['Hart', 'PERSON'],
  ["Billboard, Pink's Beautiful Trauma Tour", 'WORK_OF_ART'],
  ['10th', 'ORDINAL'],
  ['more than $397 million', 'MONEY'],
  ['Cable News Network', 'ORG'],
  ['Turner Broadcasting System,', 'ORG'],
  ['All Rights Reserved', 'WORK_OF_ART'],
  ['CNN', 'ORG'],
  ['CNN', 'ORG']],
 [['Deval Patrick', 'PERSON'],
  ['CNN', 'ORG'],
  ['Massachusetts', 'GPE'],
  ['Deval Patrick', 'PERSON'],
  ['Wednesday', 'DATE'],
  ['two', 'CARDINAL'],
  ['flux Wednesday', 'DATE'],
  

<IPython.core.display.Javascript object>

### For every document in the corpus, iterate over every sentence, extract any SVO triples, print them, and compare.

In [62]:
results = []
for doc in docs:
    for sent in spacy_doc.sents:
        svo = textacy.extract.subject_verb_object_triples(sent)
        results += svo

results = list(set(results))

<IPython.core.display.Javascript object>

In [63]:
results

[(protesters, to cross, it),
 (government, unveiled, range),
 (We, warn, protesters),
 (what, is believed, be),
 (state media, warning, protesters),
 (escalation, raised, questions),
 (protesters, seek, to breach),
 (We, have had, talking),
 (thousands, prepared, to face),
 (who, are coerced, be),
 (level, has reached, heights),
 (those, continued, to pour),
 (they, had evacuated, students),
 (police, fired, gas),
 (government supporters, hurling, bricks),
 (Adam Ni, told, CNN),
 (protesters, continued, to pour),
 (run television channel, issued, editorial),
 (MTR, continued, to suspend),
 (he, began, clearing),
 (country, will never accept, situation),
 (We, have had, persuasion),
 (We, have had, warnings),
 (students, have been asked, to leave),
 (It, echoed, editorial),
 (Student protesters, fortify, campus occupations),
 (protests, have taken, turn),
 (you, can make, call),
 (protesters, barricaded, themselves),
 (police, have appeared, reticent),
 (protesters, hurling, bricks),
 (

<IPython.core.display.Javascript object>